# Testing Robustness of Feature Weighting on Water Data

## Imports

In [ ]:
import pathlib
import re

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from ase.io import read as ase_read

from dadapy.feature_weighting import FeatureWeighting

## Import Data

Large SOAP datasets and especially ACSFs are somewhat slow to calculate, so I mostly just run this on our local cluster and then load from numpy files.
To recalculate the descriptors, use `make_descriptors.py`.

In [ ]:
data_dir = pathlib.Path("../data").resolve()

In [ ]:
# Get ase.Atoms objects for each liquid configuration
liquid_frames = ase_read(data_dir.joinpath("ice_in_water_data/dataset_1000_eVAng.xyz"), index=':')
n_atoms = np.sum(np.asarray([len(frame) for frame in liquid_frames], dtype=np.int16))
atom_types = np.zeros((n_atoms), dtype=np.int8)

# Collect some metadata, like how many atoms/config, atoms in total and which atom is even an oxygen.
counter = 0
for frame in liquid_frames:
    atom_types[counter:counter+len(frame)] = frame.get_atomic_numbers()
    counter+=len(frame)
is_o = atom_types==8
is_h = np.logical_not(is_o)

print(f"Found {np.count_nonzero(is_o)} Oxygen atoms and {np.count_nonzero(is_h)} Hydrogen atoms.")

In [ ]:
# Get new descriptors from file or laod ase.Atoms objects and recalculate
# Recalculation here works best when just getting SOAPs, ACSF takes a little long
average_soap = np.load(data_dir.joinpath("ice_in_water_data/average_soap_rcut6_nmax6_lmax6_sigma03.npy"))
atomic_soap = np.load(data_dir.joinpath("ice_in_water_data/singleatom_soap_rcut6_nmax6_lmax6_sigma03.npy"))
print("Fetched computed atomic SOAP descriptors for %u configurations and with %u features each."%atomic_soap.shape)
print("Fetched computed global SOAP descriptors for %u configurations and with %u features each."%average_soap.shape)

# The file format of the input file the descriptors are calculated from is 54 solid, 1000 liquid
# So we can just get the liquid configurations by getting the number of atoms n_atoms in the liquid configurations
# From the end of the decriptor matrix
liquid_atomic_soap = atomic_soap[-n_atoms:, :].copy()

In [ ]:
average_acsf = np.asarray(np.load(data_dir.joinpath("ice_in_water_data/average_acsf_rcut6_gridsearch_bohr_lambda.npy")), dtype=np.float32)
atomic_acsf = np.asarray(np.load(data_dir.joinpath("ice_in_water_data/singleatom_acsf_rcut6_gridsearch_bohr_lambda.npy")), dtype=np.float32)
liquid_atomic_acsf = atomic_acsf[-n_atoms:, :].copy()
print("Fetched computed atomic SOAP descriptors for %u configurations and with %u features each."%atomic_acsf.shape)
print("Fetched computed liquid atomic SOAP descriptors for %u configurations and with %u features each."%liquid_atomic_acsf.shape)
print("Fetched computed global SOAP descriptors for %u configurations and with %u features each."%average_acsf.shape)

### Post Processing of Input Data

In [ ]:
descriptors = [average_soap, atomic_soap, liquid_atomic_soap, average_acsf, atomic_acsf, liquid_atomic_acsf]
for desc in descriptors:
    desc /= np.linalg.norm(desc, axis=-1)[:, np.newaxis]
average_soap, atomic_soap, liquid_atomic_soap, average_acsf, atomic_acsf, liquid_atomic_acsf = descriptors

# apparently atomic acsf sometimes become nan, set to 0
nan_frames = np.argwhere(np.isnan(atomic_acsf))[:, 0]
print("Removing %u nan frames in atomic acsf"%(len(np.unique(nan_frames))))
atomic_acsf[nan_frames, :] = 0.

nan_frames = np.argwhere(np.isnan(liquid_atomic_acsf))[:, 0]
print("Removing %u nan frames in liquid atomic acsf"%(len(np.unique(nan_frames))))
liquid_atomic_acsf[nan_frames, :] = 0.

### Defining Input and Target Data

`target_data` is the space which delivers the target ranks, `input_space` is the space for which the gammas should be optimised.

In [ ]:
rng = np.random.default_rng()
# random_selection = rng.choice(liquid_atomic_soap.shape[0], 300)
target_data = liquid_atomic_soap[::500]
input_space = liquid_atomic_acsf[::500]

fw_input = FeatureWeighting(coordinates=input_space, maxk=input_space.shape[0]-1, verbose=True)
fw_target = FeatureWeighting(coordinates=target_data, maxk=target_data.shape[0]-1, verbose=True)

if True:
    stds = np.std(input_space, axis=0)
    stds[stds==0.] = 1.
    standardised_input = input_space/stds[np.newaxis, :]
    # dist_matrix = kimb.compute_dist_matrix(data=target_data.astype(np.double), period=np.zeros((target_data.shape[-1])))
    # truth_ranks = stats.rankdata(dist_matrix, method='average', axis=1).astype(int, copy=False)

print(f"Working on ground truth shaped {target_data.shape} and optimising space of shape {input_space.shape}")

## Lasso Optimisation

### Find Optimal Learning Rate

In [ ]:
initial_gammas = np.ones((input_space.shape[-1],), dtype=np.double)
lr_list = np.logspace(-2, 2, 10)

opt_l_rate = fw_input.return_optimal_learning_rate(
    target_data=fw_target, initial_weights=initial_gammas, lambd=None, 
    n_epochs=50, decaying_lr=True, 
    n_samples=154, trial_learning_rates=lr_list,
)

lr_kernel_imbs = fw_input.history['dii_per_epoch_per_lr'][np.argwhere(opt_l_rate==lr_list)[0, 0], :]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.plot(lr_kernel_imbs, label="kernel imbalances")

ax.set_title("Loss for optimal learning rate %f"%opt_l_rate)
# ax.set_yscale('log')
ax.set_ylabel('Kernel Imbalance')
ax.set_xlabel("epoch")

plt.show()

### Optimise using Lasso Regularisation

In [ ]:
(
    num_nonzero_features,
    l1_penalties_opt_per_nfeatures,
    dii_opt_per_nfeatures, 
    weights_opt_per_nfeatures
) = fw_input.return_lasso_optimization_dii_search(
    target_data=fw_target, initial_weights=initial_gammas, lambd=None, 
    learning_rate=opt_l_rate, constrain=True, decaying_lr=True,
    n_epochs=100, refine=True
)

In [ ]:
# invert order so it has least features first
kernel_imbs = dii_opt_per_nfeatures[::-1]
lasso_gammas = weights_opt_per_nfeatures[::-1]
num_nonzero_features = num_nonzero_features[::-1]
l1_penalties_opt_per_nfeatures = l1_penalties_opt_per_nfeatures[::-1]

## I/O for Precomputed Weights

In [ ]:
mode = 'load_old'

if mode == 'load_old':
    num_nonzero_features = np.loadtxt(data_dir.joinpath('water_phase_store/robustness_num_nonzero_features.txt'))
    l1_penalties_opt_per_nfeatures = np.loadtxt(data_dir.joinpath('water_phase_store/robustness_l1_penalties_opt_per_nfeatures.txt'))
    kernel_imbs = np.loadtxt(data_dir.joinpath('water_phase_store/robustness_kernel_imbs.txt'))
    lasso_gammas = np.loadtxt(data_dir.joinpath('water_phase_store/robustness_lasso_gammas.txt'))
if mode == 'save_new':
    np.savetxt(data_dir.joinpath('water_phase_store/robustness_num_nonzero_features.txt'), num_nonzero_features)
    np.savetxt(data_dir.joinpath('water_phase_store/robustness_l1_penalties_opt_per_nfeatures.txt'), l1_penalties_opt_per_nfeatures)
    np.savetxt(data_dir.joinpath('water_phase_store/robustness_kernel_imbs.txt'), kernel_imbs)
    np.savetxt(data_dir.joinpath('water_phase_store/robustness_lasso_gammas.txt'), weights_opt_per_nfeatures)

## Robustness

### Matching weights
Take $L_1$ penalty from weighting with certain number of features and test if same features (or maybe only with a similar value of DII) are selected by the algorithm.

In [ ]:
n_true_feat = 10
true_weights = lasso_gammas[n_true_feat-1]
true_l1 = l1_penalties_opt_per_nfeatures[n_true_feat-1]

assert np.count_nonzero(true_weights) == n_true_feat, "Wrong number of features selected."

n_compare = 30
compare_weights = lasso_gammas[n_compare-1]
compare_l1 = l1_penalties_opt_per_nfeatures[n_compare-1]

assert np.count_nonzero(compare_weights) == n_compare, "Wrong number of features selected."

print(kernel_imbs.shape)
plt.plot(kernel_imbs)
plt.xlim(0, 25)
plt.show()

plt.plot(l1_penalties_opt_per_nfeatures)
plt.yscale('log')
plt.show()

In [ ]:
n_datas = [50, 70, 100, 150, 200, 250, 300, 350]

weights_per_ndata = []
dii_compare = []

for n_data in n_datas:
    test_indices = rng.choice(liquid_atomic_soap.shape[0], n_data)

    fw_input = FeatureWeighting(coordinates=liquid_atomic_acsf[test_indices], maxk=n_data-1, verbose=True)
    fw_target = FeatureWeighting(coordinates=liquid_atomic_soap[test_indices], maxk=n_data-1, verbose=True)

    initial_gammas = np.ones((input_space.shape[-1],), dtype=np.double)
    weights = fw_input.return_weights_optimize_dii(
        target_data=fw_target, initial_weights=initial_gammas, lambd=None,
        learning_rate=None, l1_penalty=compare_l1,
        constrain=True, decaying_lr=True,
        n_epochs=100
    )
    print(f"Optimised weights for {n_data} data points.")
    print(f"DII: {fw_input.history['dii_per_epoch'][-1]}")
    dii_compare.append(fw_input.history['dii_per_epoch'][-1])
    
    weights_per_ndata.append(weights)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3))

ax.plot([np.count_nonzero(weights) for weights in weights_per_ndata], label="kernel imbalances")
plt.show()

where_true = np.argwhere(true_weights)
for weights in weights_per_ndata:
    where_weights = np.argwhere(weights)
    
    print(f"Found {np.count_nonzero(weights)} features.")
    print(f"{len(np.intersect1d(where_true, where_weights))} are in top 10.")


### Training multiple Networks

As it turns out, collecting weights for a different number of features only partially reproduces the weights gathered by the "full" dataset of 380 features.
Therefore, run the DII optimisation on multiple different number of features and reproduce predictions there to see differences.

In [ ]:
mode = 'save_new'

n_datas = np.logspace(6.5, 8, 4, base=2).astype(np.int16)
print(f'Running for {n_datas} number of points')

dii_per_nfeatures_ndata = []
weights_per_nfeatures_ndata = []
for n_data in n_datas:
    if mode == 'save_new':
        test_indices = rng.choice(liquid_atomic_soap.shape[0], n_data)

        fw_input = FeatureWeighting(coordinates=liquid_atomic_acsf[test_indices], maxk=n_data-1, verbose=True)
        fw_target = FeatureWeighting(coordinates=liquid_atomic_soap[test_indices], maxk=n_data-1, verbose=True)

        (
            num_nonzero_features,
            l1_penalties_opt_per_nfeatures,
            dii_opt_per_nfeatures, 
            weights_opt_per_nfeatures
        ) = fw_input.return_lasso_optimization_dii_search(
            target_data=fw_target, initial_weights=initial_gammas, lambd=None, 
            learning_rate=None, constrain=True, decaying_lr=True,
            n_epochs=100, refine=True
        )
        dii_per_nfeatures = dii_opt_per_nfeatures[::-1]
        weights_per_nfeatures = weights_opt_per_nfeatures[::-1]
        
        np.savetxt(data_dir.joinpath(f'water_phase_store/dii_per_nfeatures_ndata_{n_data}.txt'), dii_per_nfeatures)
        np.savetxt(data_dir.joinpath(f'water_phase_store/weights_per_nfeatures_ndata_{n_data}.txt'), weights_per_nfeatures)
    elif mode == 'load_old':
        dii_per_nfeatures = np.loadtxt(data_dir.joinpath(f'water_phase_store/dii_per_nfeatures_ndata_{n_data}.txt'))
        weights_per_nfeatures = np.loadtxt(data_dir.joinpath(f'water_phase_store/weights_per_nfeatures_ndata_{n_data}.txt'))
    dii_per_nfeatures_ndata.append(dii_per_nfeatures)
    weights_per_nfeatures_ndata.append(weights_per_nfeatures)

### Results of Training

In [ ]:
from glob import glob
import re

target_dir = data_dir.joinpath("n2p2_fitting/240205_training_out/")

results_dict = {}
for logfile in target_dir.glob('*.log'):
    with open(logfile, 'r') as f:
        content = f.read()
        runtype = re.search("231213_pot_acsf(.*)_hartbohr_scaleunits_bcdata_lambda", content).group(1)
        memory_kb = re.search("\s([0-9]*)maxresident", content).group(1)
        runtime_s = re.search("([0-9]*)user", content).group(1)
        
        pot_dir = target_dir.joinpath("231213_pot_acsf"+runtype+"_hartbohr_scaleunits_bcdata_lambda/")
        all_errors = np.loadtxt(pot_dir.joinpath('learning-curve.out'))
        all_errors[:, 1:9] *= 27.211386245988
        all_errors[:, 9:13] *= 51.421 # Hartree/Bohr to eV/Angstrom conversion
        rmse_ftest_evA = all_errors[-1, 12]
        
        results_dict[runtype] = {"memory": int(memory_kb), "runtime": float(runtime_s), "rmse": rmse_ftest_evA}
print(results_dict)

In [ ]:
import re

target_dir = data_dir.joinpath("n2p2_fitting/240724_pot_si_ndatas_out/")

results_dict_si = {}
for potdir in target_dir.glob('ndata*'):
    ndata = re.match(r'ndata\_(\d+)\_nfeat\_10', potdir.name).group(1)
    all_errors = np.loadtxt(potdir.joinpath('learning-curve.out'))
    all_errors[:, 1:9] *= 27.211386245988
    all_errors[:, 9:13] *= 51.421 # Hartree/Bohr to eV/Angstrom conversion
    rmse_ftest_evA = all_errors[-1, 12]
    
    results_dict_si[ndata] = {"rmse": rmse_ftest_evA}
print(results_dict_si)

rmses_res = []
ndatas_res = []
for ndata, res in results_dict_si.items():
    ndatas_res.append(int(ndata))
    rmses_res.append(res['rmse'])

sorter_ = np.argsort(ndatas_res)
ndatas_res = np.array(ndatas_res)[sorter_]
rmses_res = np.array(rmses_res)[sorter_]

In [ ]:
import matplotlib.ticker as ticker

plt.rcParams.update(plt.rcParamsDefault)
fontsize = 12

fig, ax = plt.subplots(1, 1, figsize=(5, 3))

ax.plot(ndatas_res, rmses_res, label='SI')
ax.plot(384, results_dict['10']['rmse'], 'ro', label=r'$DII$')
ax.plot(384, results_dict['10rand']['rmse'], 'rs', label=r'random')
ax.set_xscale('log')

ax.xaxis.set_major_formatter(ticker.NullFormatter())

ax.tick_params(axis='x', which='both', bottom=False, labelbottom=False)
ax.set_xticks(ndatas_res, labels=[6.5, 7, 7.5, 8], minor=False, fontsize=fontsize)
ax.tick_params(axis='x', which='major', bottom=True, labelbottom=True)

ax.set_ylabel("Test Force RMSE [eV/A]", fontsize=fontsize)
ax.set_xlabel(r"Number of Data Points in $DII$ selection" , fontsize=fontsize)
ax.legend(fontsize=fontsize)

fig.savefig('rmse_vs_ndata_dii.png', dpi=300, format='png', bbox_inches='tight')

plt.show()